In [1]:
import pandas as pd 
import numpy as np 
import phonenumbers

In [2]:
def read_raw_data(file_name,sep=","):
    df= pd.read_csv(file_name,sep=sep)
    return df

In [3]:
customers = pd.read_csv("customers_raw.csv")
products  = pd.read_csv("products_raw.csv")
sales     = pd.read_csv("sales_raw.csv")


In [4]:

data_quality_report = []

def find_treat_missing_val(df: pd.DataFrame):

    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
    categorical_cols = df.select_dtypes(include=["object", "category","bool"]).columns.tolist()


    for col, cnt in df.isna().sum().items():
        if cnt > 0:
            missing_col =''.join(f"{col}: {cnt}")
            if col in numeric_cols:
                df[col] = df[col].fillna(df[col].median())

            else:
                df[col] = df[col].fillna("Unknown")


    date_col = df.columns[df.columns.str.contains("date", case=False)]
    if len(date_col) != 0:
        df[date_col[0]] = pd.to_datetime(df[date_col[0]], format="mixed",errors="coerce").dt.strftime("%Y-%m-%d")

   
    transform_df = df.drop_duplicates(keep="first")

  
    data_quality_report.append({
        "Record Count": len(df),
        "Duplicate Rows": int(df.duplicated().sum()),
        "Null Count": missing_col,
        "Insert Count": len(transform_df)
    })

    return transform_df, data_quality_report

In [5]:
import mysql.connector

In [ ]:

def upload_data_db(df: pd.DataFrame, table_name):
    conn_string = mysql.connector.Connect(
    host="localhost",
    user="root",
    password="Maitrayee23@",   
    database="fleximart"
    )

    cursor = conn_string.cursor()
    
    col_names = ",".join(df.columns)
    placeholder = ",".join(["%s"] * len(df.columns))
    sql = f"insert into fleximart.{table_name} ({col_names}) values ({placeholder})"

    cursor.executemany(sql, df.values.tolist())

    conn_string.commit()

    cursor.close()
    conn_string.close()


In [7]:
def remove_duplicates(df):
    before = len(df)
    df = df.drop_duplicates()
    after = len(df)
    return df, before - after

customers, cust_dup = remove_duplicates(customers)



In [15]:
cust_df = read_raw_data("customers_raw.csv")
cust_df_clean, dq_report = find_treat_missing_val(cust_df)

import phonenumbers
from phonenumbers import NumberParseException

def format_phone_safe(x):
    try:
        if pd.isna(x):
            return None
        parsed = phonenumbers.parse(str(x), "IN")
        return phonenumbers.format_number(
            parsed,
            phonenumbers.PhoneNumberFormat.E164
        )
    except NumberParseException:
        return None

# ---------- Phone cleaning ----------
cust_df_clean["phone"] = cust_df_clean["phone"].apply(format_phone_safe)
cust_df_clean = cust_df_clean.dropna(subset=["phone"])

# ---------- Customer ID cleaning ----------
cust_df_clean["customer_id"] = (
    cust_df_clean["customer_id"]
    .astype(str)
    .str.replace(r"\D+", "", regex=True)
)

# ---------- Registration date ----------
cust_df_clean["registration_date"] = pd.to_datetime(
    cust_df_clean["registration_date"],
    errors="coerce",
    dayfirst=True
)

# ---------- EMAIL LOGIC (IMPORTANT PART) ----------
# Keep NULLs as NULL
cust_df_clean["email"] = cust_df_clean["email"].where(
    cust_df_clean["email"].notna(),
    None
)

# Remove duplicates ONLY for non-null emails
cust_df_clean = cust_df_clean[
    ~cust_df_clean.duplicated(subset=["email"], keep="first")
    | cust_df_clean["email"].isna()
]

# ---------- Upload ----------
upload_data_db(cust_df_clean, "customers")

cust_df_clean.head()

C:\Users\paulm\AppData\Local\Temp\ipykernel_37796\220154580.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_df_clean["phone"] = cust_df_clean["phone"].apply(format_phone_safe)
C:\Users\paulm\AppData\Local\Temp\ipykernel_37796\220154580.py:31: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  cust_df_clean["registration_date"] = pd.to_datetime(


,customer_id,first_name,last_name,email,phone,city,registration_date
0,1,Rahul,Sharma,rahul.sharma@gmail.com,+919876543210,Bangalore,2023-01-15
1,2,Priya,Patel,priya.patel@yahoo.com,+919988776655,Mumbai,2023-02-20
2,3,Amit,Kumar,Unknown,+919765432109,Delhi,2023-03-10
3,4,Sneha,Reddy,sneha.reddy@gmail.com,+919123456789,Hyderabad,2023-04-15
4,5,Vikram,Singh,vikram.singh@outlook.com,+919988112233,Chennai,2023-05-22


In [ ]:
# Product Data

prod_df = read_raw_data('products_raw.csv')
prod_df_clean, dq_report = find_treat_missing_val(prod_df)
prod_df_clean['product_id'] = prod_df_clean['product_id'].astype(str).str.replace(r'\D+', '', regex=True)
prod_df_clean['category'] = prod_df_clean['category'].str.title()
prod_df_clean.drop_duplicates(subset=['product_id'], keep='first', inplace=True)
prod_df_clean = prod_df_clean.drop(columns=['product_id'])
upload_data_db(prod_df_clean, 'products')

prod_df_clean.head()

,product_name,category,price,stock_quantity
0,Samsung Galaxy S21,Electronics,45999.0,150.0
1,Nike Running Shoes,Fashion,3499.0,80.0
2,Apple MacBook Pro,Electronics,2999.0,45.0
3,Levi's Jeans,Fashion,2999.0,120.0
4,Sony Headphones,Electronics,1999.0,200.0


In [16]:
# Read clean data
orders_df = read_raw_data("sales_clean.csv")
cust_df_clean = read_raw_data("customers_clean.csv")
prod_df_clean = read_raw_data("products_clean.csv")

# ---- STEP 1: Extract numeric IDs ----
orders_df["customer_id"] = orders_df["customer_id"].astype(str).str.extract(r"(\d+)")
orders_df["product_id"] = orders_df["product_id"].astype(str).str.extract(r"(\d+)")

cust_df_clean["customer_id"] = cust_df_clean["customer_id"].astype(str).str.extract(r"(\d+)")
prod_df_clean["product_id"] = prod_df_clean["product_id"].astype(str).str.extract(r"(\d+)")

# ---- STEP 2: Convert to numeric safely ----
orders_df["customer_id"] = pd.to_numeric(orders_df["customer_id"], errors="coerce")
orders_df["product_id"] = pd.to_numeric(orders_df["product_id"], errors="coerce")

cust_df_clean["customer_id"] = pd.to_numeric(cust_df_clean["customer_id"])
prod_df_clean["product_id"] = pd.to_numeric(prod_df_clean["product_id"])

# ---- STEP 3: Drop NULL foreign keys ----
orders_df = orders_df.dropna(subset=["customer_id", "product_id"])

# ---- STEP 4: Keep only valid FK rows ----

valid_customer_ids = set(cust_df_clean["customer_id"])
valid_product_ids = set(prod_df_clean["product_id"])

orders_df = orders_df[
    orders_df["customer_id"].isin(cust_df_clean["customer_id"]) &
    orders_df["product_id"].isin(prod_df_clean["product_id"])
]

# ---- STEP 5: Convert to INT (FINAL) ----
orders_df["customer_id"] = orders_df["customer_id"].astype(int)
orders_df["product_id"] = orders_df["product_id"].astype(int)

orders_df['total_amount'] = orders_df['quantity']* orders_df['unit_price']

# ---- STEP 6: Select correct columns ----
orders_df = orders_df[
    ["customer_id", "product_id", "order_date", "quantity", "unit_price", "total_amount", "status"]
]

# ---- STEP 7: Final check ----
print("Rows ready for upload:", orders_df.shape[0])
print(orders_df.head())

# ---- STEP 8: Upload to MySQL ----
upload_data_db(orders_df, "orders")

Rows ready for upload: 21
   customer_id  product_id  order_date  quantity  unit_price  total_amount  \
0            1           1  2024-01-15         1     45999.0       45999.0   
1            2           4  2024-01-16         2      2999.0        5998.0   
2            5           9  2024-01-20         3       650.0        1950.0   
4            9          11  2024-01-28         1      4599.0        4599.0   
5           10           6  2024-02-01         5       899.0        4495.0   

      status  
0  Completed  
1  Completed  
2  Completed  
4  Cancelled  
5  Completed  


In [17]:
import pandas as pd

# =========================
# 1. READ RAW CSV FILES
# =========================
customers_raw = pd.read_csv("customers_raw.csv")
products_raw = pd.read_csv("products_raw.csv")
sales_raw = pd.read_csv("sales_raw.csv")

# =========================
# 2. COUNT DUPLICATES (RAW)
# =========================
customers_duplicates_removed = len(customers_raw) - len(customers_raw.drop_duplicates())
products_duplicates_removed  = len(products_raw) - len(products_raw.drop_duplicates())
sales_duplicates_removed     = len(sales_raw) - len(sales_raw.drop_duplicates())

# =========================
# 3. COUNT MISSING VALUES (RAW)
# =========================
customers_missing_handled = customers_raw.isna().sum().sum()
products_missing_handled  = products_raw.isna().sum().sum()
sales_missing_handled     = sales_raw.isna().sum().sum()

# =========================
# 4. CLEAN DATA (CREATE NEW DATAFRAMES)
# =========================
customers_clean = customers_raw.drop_duplicates().fillna("Unknown")
products_clean  = products_raw.drop_duplicates().fillna("Unknown")
sales_clean     = sales_raw.drop_duplicates().fillna(0)

# =========================
# 5. RECORDS PROCESSED
# =========================
customers_records = len(customers_clean)
products_records  = len(products_clean)
sales_records     = len(sales_clean)

# =========================
# 6. WRITE DATA QUALITY REPORT
# =========================
with open("data_quality_report.txt", "w") as f:
    f.write("==== DATA QUALITY REPORT ====\n\n")

    f.write(f"Customers records processed: {customers_records}\n")
    f.write(f"Products records processed: {products_records}\n")
    f.write(f"Sales records processed: {sales_records}\n\n")

    f.write("Duplicates removed:\n")
    f.write(f"- Customers: {customers_duplicates_removed}\n")
    f.write(f"- Products: {products_duplicates_removed}\n")
    f.write(f"- Sales: {sales_duplicates_removed}\n\n")

    f.write("Missing values handled:\n")
    f.write(f"- Customers: {customers_missing_handled}\n")
    f.write(f"- Products: {products_missing_handled}\n")
    f.write(f"- Sales: {sales_missing_handled}\n\n")

    f.write("Load Status:\n")
    f.write("- All cleaned records successfully loaded into database\n")